# Books : Visualisation and recommendation
[Kaggle | Books : Visualisation and recommendation](https://www.kaggle.com/code/nayansakhiya/books-visualisation-and-recommendation)   
[Kaggle | Bookcrossing EDA & Visualization (Plotly)](https://www.kaggle.com/code/sanzharsamatuly/bookcrossing-eda-visualization-plotly)
### DataSet 
- [goodbooks-10k](https://www.kaggle.com/datasets/zygmunt/goodbooks-10k)
- [Book-Crossing: User review ratings](https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset?select=Book+reviews)
---
# 캐굴캐굴 필사 스터디 1주차
#### 1주차 학습 기간 : 23.01.28 ~ 23.02.04   
주제 : EDA    
해당 데이터 셋에서 책 제목, 저자, 출판사, 사용자 및 등급에 대한 정보를 확인하여 사용자가 어떤 책을 읽고 있는지 인사이트를 얻어낸다.     
1. About Dataset
2. Data Visualization 
    

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import re
import string

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import plotly.express as ex
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
import plotly_express as px
%matplotlib inline


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sanga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


- pandas : data processing, CSV file I/O
- numpy : 선형대수

In [2]:
import os

root_dir = "C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset"
for (root, dirs, files) in os.walk(root_dir):
    print("# root : " + root)
    if len(dirs) > 0:
        for dir_name in dirs:
            print("dir: " + dir_name)

    if len(files) > 0:
        for file_name in files:
            print("file: " + file_name)

# root : C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset
dir: goodbooks-10
dir: reviews
dir: summary
# root : C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\goodbooks-10
file: books.csv
file: book_tags.csv
file: ratings.csv
file: tags.csv
file: to_read.csv
# root : C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\reviews
file: BX-Book-Ratings.csv
file: BX-Users.csv
file: BX_Books.csv
# root : C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\summary
file: Preprocessed_data.csv


In [3]:
for dirname, _, filenames in os.walk('C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\goodbooks-10\books.csv
C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\goodbooks-10\book_tags.csv
C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\goodbooks-10\ratings.csv
C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\goodbooks-10\tags.csv
C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\goodbooks-10\to_read.csv
C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\reviews\BX-Book-Ratings.csv
C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\reviews\BX-Users.csv
C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\reviews\BX_Books.csv
C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\summary\Preprocessed_data.csv


In [4]:
from glob import glob

print(glob('C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset/*'))

['C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\\goodbooks-10', 'C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\\reviews', 'C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\\summary']


### Read .csv data 

In [5]:
# goodreads data
goodbooks_path = 'C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset\goodbooks-10'
books_data = pd.read_csv(f'{goodbooks_path}/books.csv',error_bad_lines = False)
tags_data = pd.read_csv(f'{goodbooks_path}/book_tags.csv')
ratings_data = pd.read_csv(f'{goodbooks_path}/ratings.csv')
book_tags = pd.read_csv(f'{goodbooks_path}/tags.csv')

# book crossing data
book_path = 'C:/Users/sanga/Desktop/kaggul_kaggul_study/Study/goodbooks-10k/dataset/reviews'

user_cols = ['user_id', 'location', 'age']
cross_users_data = pd.read_csv(f'{book_path}/BX-Users.csv', sep=';', names=user_cols, encoding='latin-1', low_memory=False, skiprows=1)

book_cols = ['isbn','book_title' ,'book_author','year_of_publication', 'publisher', 'img_s', 'img_m', 'img_l']
cross_books_data = pd.read_csv(f'{book_path}/BX_Books.csv', sep=';', names=book_cols, encoding='latin-1', low_memory=False, skiprows=1)

rating_cols = ['user_id', 'isbn', 'rating']
cross_ratings_data = pd.read_csv(f'{book_path}/BX-Book-Ratings.csv', sep=';', names=rating_cols, encoding='latin-1', low_memory=False, skiprows=1)

C:\Users\sanga\AppData\Local\Temp\ipykernel_11892\4274378275.py:3: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.





In [6]:
books_data.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [7]:
cross_users_data.head()

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
cross_ratings_data.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [9]:
cross_books_data.head()

,isbn,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### Drop unnecessary data

In [10]:
books_data = books_data.drop(columns=['id', 'best_book_id', 'work_id', 'isbn', 'isbn13', 'title','work_ratings_count',
                                   'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 
                                    'image_url','small_image_url'])

In [11]:
books_data = books_data.dropna()
cross_books_data = cross_books_data.drop(columns=['img_s', 'img_m', 'img_l'])

### Drop Duplicates from all the dataset


In [12]:
ratings_data = ratings_data.sort_values("user_id")

ratings_data.drop_duplicates(subset =["user_id","book_id"], keep = False, inplace = True) 
books_data.drop_duplicates(subset='original_title',keep=False,inplace=True)
book_tags.drop_duplicates(subset='tag_id',keep=False,inplace=True)
tags_data.drop_duplicates(subset=['tag_id','goodreads_book_id'],keep=False,inplace=True)
cross_ratings_data.drop_duplicates(subset =["user_id","isbn"], keep = False, inplace = True) 
cross_books_data.drop_duplicates(subset='book_title',keep=False,inplace=True)

### Clean the text

In [13]:
def clean_text(text):
    '''Make text lowercase,
       remove text in square brackets,
       remove links,remove punctuation
       and remove words containing numbers.
    '''
    
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [14]:
cross_books_data['book_title'] = cross_books_data['book_title'].apply(lambda x:clean_text(x))

In [15]:
merge_data = pd.merge(cross_books_data, cross_ratings_data, on='isbn')
merge_data = merge_data.sort_values('isbn', ascending=True)
merge_data.head()

,isbn,book_title,book_author,year_of_publication,publisher,user_id,rating
626418,0000913154,the way things work an illustrated encyclopedi...,C. van Amerongen (translator),1967,Simon & Schuster,171118,8
587195,0001010565,mogs christmas,Judith Kerr,1992,Collins,209516,0
587194,0001010565,mogs christmas,Judith Kerr,1992,Collins,86123,0
441049,0001046713,twopence to cross the mersey,Helen Forrester,1992,HarperCollins Publishers,196149,0
263949,000104687X,ts eliot reading the wasteland and other poems,T.S. Eliot,1993,HarperCollins Publishers,23902,6


In [16]:
books_data.head()

,book_id,books_count,authors,original_publication_year,original_title,language_code,average_rating,ratings_count
0,2767052,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653
1,3,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,eng,4.44,4602479
3,2657,487,Harper Lee,1960.0,To Kill a Mockingbird,eng,4.25,3198671
4,4671,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,eng,3.89,2683664
5,11870085,226,John Green,2012.0,The Fault in Our Stars,eng,4.26,2346404


## Data Visualization 
### Good Reads Visualisation
- goodbooks dataset 설명 

```
There have been good datasets for movies (Netflix, Movielens) and music (Million Songs) recommendation, but not for books. That is, until now.

This dataset contains ratings for ten thousand popular books. As to the source, let's say that these ratings were found on the internet. Generally, there are 100 reviews for each book, although some have less - fewer - ratings. Ratings go from one to five.

Both book IDs and user IDs are contiguous. For books, they are 1-10000, for users, 1-53424. All users have made at least two ratings. Median number of ratings per user is 8.

There are also books marked to read by the users, book metadata (author, year, etc.) and tags.
```

In [17]:
top_rated = books_data.sort_values('average_rating', ascending=False)
top30 = top_rated[:30]

fig = px.bar(top30, x="average_rating", y="original_title", title='Top 30 Rated Books and Their Ratings',
             orientation='h', color='original_title', width=1500, height=700)
fig.show()

In [18]:
fig = px.treemap(top30, path=['original_title'], values='average_rating',title='Top 30 Rated Books', width=1000, height=700)
fig.show()

In [19]:
top_popular = books_data.sort_values('ratings_count', ascending=False)
top30_book = top_popular[:30]
fig = px.bar(top30_book, x="ratings_count", y="original_title", title='Top 30 Popular Books', orientation='h', color='original_title',
             width=1500, height=700)
fig.show()

In [20]:
fig = px.treemap(top30_book, path=['original_title'], values='ratings_count',title='Top 30 Popular Books', width=1000, height=700)
fig.show()

In [21]:
top_authors = top_rated[:50]
fig = px.treemap(top_authors, path=['authors'], values='average_rating',title='Popular Authors', width=1000, height=700)
fig.show()

### Bookcrossing EDA
- bookcrossing이란? 
    - 공공장소에 책을 남겨 놓아 다른 사람들도 그 책을 읽을 수 있도록 하는 일
- 책을 읽은 사람이 리뷰를 남겨놓고 점수를 준 데이터를 모아둔 것 같음 
    - 가장 많은 리뷰, 높은 점수 / 적은 리뷰, 적은 점수를 받은 책은 어떤 것이 있는지 알아보자!

데이터 셋 설명
```
1. Books
ISBN: book's unique id
Book-Title: Title of book
Book-Author: Author of book
Publisher: Organization that published book
Year-Of-Publication: published year
Image-URL-S: url of book's image small size
Image-URL-M: url of book's image medium size
Image-URL-L: url of book's image large size

2. Users
User-ID: unique id for user
Location: physicall address of user (city, state, country)
Age: age of this user

3. Book-Ratings:
User-ID: id of user who rated
ISBN: id of book that was rated
Book-Rating: score that user gives to book (from 1 to 10)
```

In [22]:
merge_data.head()

,isbn,book_title,book_author,year_of_publication,publisher,user_id,rating
626418,0000913154,the way things work an illustrated encyclopedi...,C. van Amerongen (translator),1967,Simon & Schuster,171118,8
587195,0001010565,mogs christmas,Judith Kerr,1992,Collins,209516,0
587194,0001010565,mogs christmas,Judith Kerr,1992,Collins,86123,0
441049,0001046713,twopence to cross the mersey,Helen Forrester,1992,HarperCollins Publishers,196149,0
263949,000104687X,ts eliot reading the wasteland and other poems,T.S. Eliot,1993,HarperCollins Publishers,23902,6


### Top Years of Publishing

In [23]:
# Top Years of Publishing
cross_typ = merge_data['year_of_publication'].value_counts().reset_index()
cross_typ.columns = ['value', 'count']
cross_typ['value'] = cross_typ['value'].astype(str) +' year'
cross_typ = cross_typ.sort_values('count')
fig = px.bar(cross_typ.tail(50), x="count", y="value", title='Top 20 Years of Publishing', orientation='h', color='value',
             width=1000, height=700)
fig.show()

- 2000년 , 2002 ~ 2003년 출판된 책이 가장 많이 읽힘 

In [24]:
merge_data

,isbn,book_title,book_author,year_of_publication,publisher,user_id,rating
626418,0000913154,the way things work an illustrated encyclopedi...,C. van Amerongen (translator),1967,Simon & Schuster,171118,8
587195,0001010565,mogs christmas,Judith Kerr,1992,Collins,209516,0
587194,0001010565,mogs christmas,Judith Kerr,1992,Collins,86123,0
441049,0001046713,twopence to cross the mersey,Helen Forrester,1992,HarperCollins Publishers,196149,0
263949,000104687X,ts eliot reading the wasteland and other poems,T.S. Eliot,1993,HarperCollins Publishers,23902,6
...,...,...,...,...,...,...,...
413438,B0001FZGTO,ill let you go a novel,Bruce Wagner,2002,Villard,271705,0
575503,B0001I1KOG,new york public library literature companion,New York Public Library,2001,Free Press,148258,10
472829,B00029DGGO,good wife strikes back the,Elizabeth Buchan,0,Viking Adult,100088,0
543768,B0002JV9PY,the blockade runners,Jules Verne,0,Digireads.com,179791,0


In [34]:
# top authors(frequency of book)
cross_author = merge_data['book_author'].value_counts().reset_index()
cross_author.columns = ['value', 'count']

cross_author['value'] = cross_author['value'].astype(str)
cross_author = cross_author.sort_values('count')

fig = px.bar(cross_author.tail(50), x="count", y="value", title='Top 20 Authors', orientation='h', color='value',
             width=1000, height=700)
fig.show()

In [26]:
top_book = merge_data['book_title'].value_counts().reset_index()
top_book.columns = ['value', 'count']
top_book['value'] = top_book['value']
top_book = top_book.sort_values('count')
fig = px.bar(top_book.tail(20), x="count", y="value", title='Top 20 Books', orientation='h', color='value',
             width=1000, height=700)
fig.show()

In [27]:
# with publishers
by_publisher = merge_data.groupby('publisher')['isbn'].count()
by_publisher = by_publisher.sort_values(ascending = False)

figure = go.Figure(data = [go.Pie(labels = by_publisher.index[:10], 
                                  values = by_publisher.values,
                                  textinfo='label+percent',
                                  title= 'Ten publsihers with most published books.' 
                                 )])

figure.show()

In [28]:
print(f'Ten most popular publishers published: {sum(by_publisher.values[:10])}, Others: {sum(by_publisher.values[11:])}')

Ten most popular publishers published: 131359, Others: 501928


In [29]:
# with book title
stopwords.words('english')
print(stopwords.words('english'))
# Here I want to split every title of book to words and check whereas this word length greater than 3 or it's not in stop words.
# If everything is fine, then I will add that word into array.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [30]:
from collections import Counter
results = Counter()
merge_data['book_title'].str.lower().str.split().apply(results.update)

626418    None
587195    None
587194    None
441049    None
263949    None
          ... 
413438    None
575503    None
472829    None
543768    None
543766    None
Name: book_title, Length: 641731, dtype: object

In [31]:
from tqdm import tqdm 
most_common = dict(results.most_common(1000))

to_replace = ['(', ')', '*', '((', '))']
own_stop_words = ['book', 'guide', 'novel']

top_words = {}

for k, v in tqdm(most_common.items()):
    
    for err in to_replace:
        k = k.replace(err, "")
        
    if (len(k) > 3) and (k not in stopwords.words()) and (k not in own_stop_words):
        top_words[k] = v

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:32<00:00, 30.65it/s]


- 책의 모든 제목을 단어로 나누고 이 단어의 길이가 3보다 크고 stopword에 포함되지 않은 단어 수집 

In [32]:
# top_words, Sort dictionary by values
top_words = dict(sorted(top_words.items(), key=lambda item: item[1])[::-1][:20])

In [33]:
figure = go.Figure(data = [go.Pie(labels = list(top_words.keys()), 
                                  values = list(top_words.values()),
                                  textinfo='label+percent',
                                  title= 'Twenty most used words in book titles.' 
                                 )])

figure.show()

- 미스테리, 시리즈, 생활, 클래식, 스토리 등등 가장 많이 사용되는 단어 20개를 시각화